In [1]:
# import Required Libraries

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Adjusting Row Column Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

In [3]:
# Loading the Data Set

movie = pd.read_csv('/content/drive/MyDrive/movie_no_year.csv')
rating = pd.read_csv('/content/drive/MyDrive/rating.csv')

In [4]:
# Merging movie and rating data sets

df = movie.merge(rating, how="left", on="movieId")

In [5]:
# Preliminary examination of the data set

def check_df(dataframe, head=5):
    print('##################### Shape #####################')
    print(dataframe.shape)
    print('##################### Types #####################')
    print(dataframe.dtypes)
    print('##################### Head #####################')
    print(dataframe.head(head))
    print('##################### Tail #####################')
    print(dataframe.tail(head))
    print('##################### NA #####################')
    print(dataframe.isnull().sum())
    print('##################### Quantiles #####################')
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(20000797, 6)
##################### Types #####################
movieId        int64
title         object
genres        object
userId       float64
rating       float64
timestamp     object
dtype: object
##################### Head #####################
   movieId      title                                       genres  userId  rating            timestamp
0        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy     3.0     4.0  1999-12-11 13:36:47
1        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy     6.0     5.0  1997-03-13 17:50:52
2        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy     8.0     4.0  1996-06-05 13:37:51
3        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy    10.0     4.0  1999-11-25 02:44:47
4        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy    11.0     4.5  2009-01-02 01:13:41
##################### Tail #####################
          movieId       

In [6]:
# We determined how many comments each film received.
comment_counts = pd.DataFrame(df["title"].value_counts())

In [7]:
comment_counts.head(20)

,count
title,
Pulp Fiction,67310
Forrest Gump,66172
"Shawshank Redemption, The",63366
"Silence of the Lambs, The",63299
Jurassic Park,59715
Star Wars: Episode IV - A New Hope,54502
Braveheart,53769
Terminator 2: Judgment Day,52244
"Matrix, The",51334


In [8]:
print(df.columns)
print(df['title'].isnull().sum())
df = df[df['title'].notna()]

Index(['movieId', 'title', 'genres', 'userId', 'rating', 'timestamp'], dtype='object')
0


In [9]:
print(comment_counts.head())
print(comment_counts.index)

                           count
title                           
Pulp Fiction               67310
Forrest Gump               66172
Shawshank Redemption, The  63366
Silence of the Lambs, The  63299
Jurassic Park              59715
Index(['Pulp Fiction', 'Forrest Gump', 'Shawshank Redemption, The', 'Silence of the Lambs, The', 'Jurassic Park', 'Star Wars: Episode IV - A New Hope', 'Braveheart', 'Terminator 2: Judgment Day', 'Matrix, The', 'Schindler's List',
       ...
       'Whitecoats', 'L'antisémite', 'Night Walker, The', 'Shadow Puppets', 'Stille nacht', 'Into the Mind', 'Road to Ruin, The', 'Prize of Peril, The (Prix du danger, Le)', 'Nukes in Space', 'Rentun Ruusu'], dtype='object', name='title', length=26216)


In [10]:
# Access the count values directly using the Series
rare_movies = comment_counts[comment_counts['count'] <= 1000].index

In [11]:
rare_movies

Index(['Ted', 'Bear, The (Ours, L')', 'Rosewood', 'One Night at McCool's', 'Marked for Death', 'Three to Tango', 'Adam's Rib', 'Frankie and Johnny', 'Stakeout', 'Someone Like You',
       ...
       'Whitecoats', 'L'antisémite', 'Night Walker, The', 'Shadow Puppets', 'Stille nacht', 'Into the Mind', 'Road to Ruin, The', 'Prize of Peril, The (Prix du danger, Le)', 'Nukes in Space', 'Rentun Ruusu'], dtype='object', name='title', length=23082)

In [12]:
common_movies = df[~df["title"].isin(rare_movies)]

In [13]:
common_movies["title"].nunique()

3134

In [14]:
df["title"].nunique()

26216

In [15]:
# Load a random sample of the rating dataset
rating_sample = rating.sample(frac=0.1, random_state=42)  # 10% sample, adjust as needed

# Merge with the movie data
df_sample = movie.merge(rating_sample, how="left", on="movieId")

# Filter for common movies
comment_counts = df_sample['title'].value_counts()
rare_movies = comment_counts[comment_counts <= 1000].index
common_movies_sample = df_sample[~df_sample['title'].isin(rare_movies)]

# Create the User-Movie DataFrame
user_movie_df = common_movies_sample.pivot_table(index="userId", columns="title", values="rating")


In [16]:
print(common_movies.head())
print(common_movies['title'].nunique())


   movieId      title                                       genres  userId  rating            timestamp
0        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy     3.0     4.0  1999-12-11 13:36:47
1        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy     6.0     5.0  1997-03-13 17:50:52
2        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy     8.0     4.0  1996-06-05 13:37:51
3        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy    10.0     4.0  1999-11-25 02:44:47
4        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy    11.0     4.5  2009-01-02 01:13:41
3134


In [17]:
user_movie_df.head()

title,10 Things I Hate About You,12 Angry Men,2001: A Space Odyssey,28 Days Later,300,A.I. Artificial Intelligence,"Abyss, The",Ace Ventura: Pet Detective,Ace Ventura: When Nature Calls,Addams Family Values,"African Queen, The",Air Force One,Airplane!,Aladdin,Alice in Wonderland,Alien,Alien: Resurrection,Aliens,Alien³ (a.k.a. Alien 3),Almost Famous,Amadeus,"Amelie (Fabuleux destin d'Amélie Poulain, Le)",American Beauty,American History X,American Pie,"American President, The",Animal House,Annie Hall,Antz,Apocalypse Now,Apollo 13,Arachnophobia,Armageddon,Army of Darkness,As Good as It Gets,Austin Powers: International Man of Mystery,Austin Powers: The Spy Who Shagged Me,Babe,Back to the Future,Back to the Future Part II,Back to the Future Part III,Bad Boys,Basic Instinct,Batman,Batman Begins,Batman Forever,Batman Returns,"Beautiful Mind, A",Beauty and the Beast,Beetlejuice,Being John Malkovich,Best in Show,Beverly Hills Cop,Beverly Hills Cop III,Big,Big Fish,"Big Lebowski, The","Birdcage, The","Birds, The",Black Hawk Down,Blade,Blade Runner,"Blair Witch Project, The",Blazing Saddles,"Blues Brothers, The",Boogie Nights,"Boot, Das (Boat, The)","Bourne Identity, The","Bourne Supremacy, The","Bourne Ultimatum, The",Bowling for Columbine,Braveheart,Brazil,"Breakfast Club, The","Bridge on the River Kwai, The",Bridget Jones's Diary,Broken Arrow,"Bug's Life, A",Butch Cassidy and the Sundance Kid,"Cable Guy, The",Cape Fear,Casablanca,Casino,Casino Royale,Casper,Cast Away,Catch Me If You Can,Charlie's Angels,Chasing Amy,Chicken Run,Chinatown,Chocolat,"Christmas Story, A",Citizen Kane,City Slickers II: The Legend of Curly's Gold,City of God (Cidade de Deus),Clear and Present Danger,Clerks,"Client, The",Cliffhanger,"Clockwork Orange, A",Close Encounters of the Third Kind,Clueless,Cocoon,Con Air,Coneheads,Congo,Conspiracy Theory,Contact,Cool Hand Luke,Crash,Crimson Tide,Crocodile Dundee,"Crouching Tiger, Hidden Dragon (Wo hu cang long)","Crow, The","Crying Game, The",Dances with Wolves,Dark City,"Dark Knight, The",Dave,Dead Man Walking,Dead Poets Society,Deep Impact,"Deer Hunter, The",Demolition Man,"Departed, The",Desperado,"Devil's Advocate, The",Die Hard,Die Hard 2,Die Hard: With a Vengeance,Dirty Dancing,Disclosure,Dogma,Donnie Brasco,Donnie Darko,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,Dragonheart,Dumb & Dumber (Dumb and Dumber),Dune,E.T. the Extra-Terrestrial,Ed Wood,Edward Scissorhands,Election,Enemy of the State,"English Patient, The",Eraser,Erin Brockovich,Eternal Sunshine of the Spotless Mind,Executive Decision,"Exorcist, The",Eyes Wide Shut,Face/Off,Fantasia,Fargo,Father of the Bride Part II,Ferris Bueller's Day Off,"Few Good Men, A",Field of Dreams,"Fifth Element, The",Fight Club,Finding Nemo,"Firm, The",First Knight,"Fish Called Wanda, A","Fly, The",Forrest Gump,Four Weddings and a Funeral,French Kiss,From Dusk Till Dawn,"Fugitive, The",Full Metal Jacket,"Full Monty, The",Galaxy Quest,"Game, The",Gattaca,Get Shorty,Ghost,Ghostbusters (a.k.a. Ghost Busters),Gladiator,Glory,"Godfather, The","Godfather: Part II, The","Godfather: Part III, The",GoldenEye,Goldfinger,Gone with the Wind,"Good Morning, Vietnam",Good Will Hunting,"Good, the Bad and the Ugly, The (Buono, il brutto, il cattivo, Il)",Goodfellas,"Goonies, The","Graduate, The",Grease,"Great Escape, The","Green Mile, The",Gremlins,Grosse Pointe Blank,Groundhog Day,Grumpier Old Men,Hamlet,Happy Gilmore,Harry Potter and the Chamber of Secrets,Harry Potter and the Goblet of Fire,Harry Potter and the Prisoner of Azkaban,Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone),Heat,Heathers,High Fidelity,Highlander,Home Alone,"Honey, I Shrunk the Kids",Hoop Dreams,Hot Shots! Part Deux,"Hudsucker Proxy, The","Hunchback of Notre Dame, The","Hunt for Red October, The","I, Robot",I.Q.,Ice Age,In the Line of Fire,Inception,"Incredibles, The",Independence Day (a.k.a. ID4),Indiana Jones and the Last Crusade,Indiana Jones and the

In [18]:
user_movie_df.shape

(126374, 470)

In [19]:
user_movie_df.columns

Index(['10 Things I Hate About You', '12 Angry Men', '2001: A Space Odyssey', '28 Days Later', '300', 'A.I. Artificial Intelligence', 'Abyss, The', 'Ace Ventura: Pet Detective', 'Ace Ventura: When Nature Calls', 'Addams Family Values',
       ...
       'Wild Wild West', 'William Shakespeare's Romeo + Juliet', 'Willy Wonka & the Chocolate Factory', 'Witness', 'Wizard of Oz, The', 'X-Files: Fight the Future, The', 'X-Men', 'X2: X-Men United', 'You've Got Mail', 'Young Frankenstein'], dtype='object', name='title', length=470)

In [45]:
movie_name1 = "Ocean's Twelve"

In [46]:
print(user_movie_df.columns)
[movie for movie in user_movie_df.columns if 'Matrix' in movie]
user_movie_df.columns = user_movie_df.columns.str.strip()


Index(['10 Things I Hate About You', '12 Angry Men', '2001: A Space Odyssey', '28 Days Later', '300', 'A.I. Artificial Intelligence', 'Abyss, The', 'Ace Ventura: Pet Detective', 'Ace Ventura: When Nature Calls', 'Addams Family Values',
       ...
       'Wild Wild West', 'William Shakespeare's Romeo + Juliet', 'Willy Wonka & the Chocolate Factory', 'Witness', 'Wizard of Oz, The', 'X-Files: Fight the Future, The', 'X-Men', 'X2: X-Men United', 'You've Got Mail', 'Young Frankenstein'], dtype='object', name='title', length=470)


In [47]:
if movie_name1 in user_movie_df.columns:
    movie_name = user_movie_df[movie_name1]
else:
    print(f"'{movie_name1}' not found in user_movie_df columns.")


'Ocean's Twelve' not found in user_movie_df columns.


In [48]:
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

,0
title,
"Nutty Professor, The",1.000000
Cliffhanger,0.746139
Disclosure,0.714964
I.Q.,0.606340
Bad Boys,0.582309
Broken Arrow,0.565322
Star Trek III: The Search for Spock,0.534792
Con Air,0.532272
Addams Family Values,0.529005


In [25]:
movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]

In [26]:
movie_name = user_movie_df[movie_name]

In [27]:
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

,0
title,
"Nutty Professor, The",1.000000
Cliffhanger,0.746139
Disclosure,0.714964
I.Q.,0.606340
Bad Boys,0.582309
Broken Arrow,0.565322
Star Trek III: The Search for Spock,0.534792
Con Air,0.532272
Addams Family Values,0.529005


In [28]:
def check_film(keyword, user_movie_df):
    return [col for col in user_movie_df.columns if keyword in col]

In [29]:
check_film("Insomnia", user_movie_df)


[]

In [42]:
# Install necessary dependencies with forced reinstallation
!pip install --ignore-installed itsdangerous blinker
!pip install --no-deps flask

from flask import Flask, render_template, request, send_from_directory
import pandas as pd
import os
import shutil
from google.colab import drive
from google.colab.output import eval_js
import re
from urllib.parse import quote

import requests

# TMDb API details
TMDB_API_KEY = "cded237d0baeec6b725cacea5d0c4a2c"
TMDB_BASE_URL = "https://api.themoviedb.org/3"
TMDB_IMAGE_BASE_URL = "https://image.tmdb.org/t/p/w500"

  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)


In [44]:
# BEST SO FAR

from urllib.parse import quote

# Đường dẫn tới poster trong Google Drive
POSTER_PATH = '/content/drive/MyDrive/posters/posters'
# Đường dẫn tới thư mục tĩnh trong Colab
STATIC_POSTER_PATH = '/content/posters/'


"""
# Tạo thư mục tĩnh nếu chưa tồn tại
os.makedirs(STATIC_POSTER_PATH, exist_ok=True)


# Sao chép các poster vào thư mục tĩnh
for filename in os.listdir(POSTER_PATH):
    if filename.endswith('.jpg'):
        shutil.copy(os.path.join(POSTER_PATH, filename), STATIC_POSTER_PATH)

"""

# Khởi tạo ứng dụng Flask
app = Flask(__name__)

# Giả sử df_sample và user_movie_df đã được định nghĩa ở đâu đó
# df_sample = pd.read_csv('path_to_movies_csv')  # Đường dẫn tới file CSV chứa thông tin phim
# user_movie_df = pd.read_csv('path_to_user_movie_csv')  # Đường dẫn tới file CSV chứa thông tin người dùng

def clean_title(title):
    # Example: strip whitespace and convert to lowercase
    return title.strip().lower()


def get_movie_info(movie_title):
    # Check for movie information in the dataset
    movie_info_df = df_sample[df_sample['title'].str.lower() == movie_title.lower()]

    # Attempt to load the local poster if available
    if not movie_info_df.empty:
        movie_info = movie_info_df.iloc[0]
        poster_filename = f"{movie_title.replace(' ', '_')}.jpg"
        local_poster_path = os.path.join(STATIC_POSTER_PATH, poster_filename)

        # Check if the local poster exists; if not, use TMDb
        if os.path.isfile(local_poster_path):
            return {
                'title': movie_info['title'],
                'genres': movie_info['genres'],
                'poster': f'/posters/{poster_filename}'
            }

    # Fetch data from TMDb if not in local or missing the local poster
    response = requests.get(
        f"{TMDB_BASE_URL}/search/movie",
        params={"api_key": TMDB_API_KEY, "query": movie_title}
    )
    data = response.json()
    if data["results"]:
        movie_data = data["results"][0]
        poster_url = f"{TMDB_IMAGE_BASE_URL}{movie_data['poster_path']}" if movie_data.get('poster_path') else None
        return {
            'title': movie_data['title'],
            'genres': "Genre data unavailable",
            'poster': poster_url or '/path/to/placeholder.jpg'  # Use a placeholder if no poster found
        }
    else:
        print(f"Movie '{movie_title}' not found on TMDb.")
        return None




# Hàm gợi ý phim dựa vào Collaborative Filtering
def recommend_movies(movie_name):
    if movie_name not in user_movie_df.columns:
        print(f"Movie '{movie_name}' not found in user_movie_df")  # Debug: Print missing movie
        return []

    movie_series = user_movie_df[movie_name]
    similar_movies = user_movie_df.corrwith(movie_series).sort_values(ascending=False).head(10)

    recommended_movies = []
    for title in similar_movies.index:
        if title != movie_name:
            info = get_movie_info(title)
            if info:
                recommended_movies.append(info)

    return recommended_movies



# Main route
@app.route('/', methods=['GET', 'POST'])
def index():
    recommendations = []
    movie_info = None
    if request.method == 'POST':
        movie_name = request.form.get('movie_name')
        print(f"Received movie name: {movie_name}")  # Debug: Print received movie name

        movie_info = get_movie_info(movie_name)
        print(f"Movie info: {movie_info}")  # Debug: Print movie info

        if movie_info:
            recommendations = recommend_movies(movie_name)
            print(f"Recommendations: {recommendations}")  # Debug: Print recommendations

    return render_template('index.html', movie_info=movie_info, recommendations=recommendations)


# Route để phục vụ các tệp tĩnh (nếu cần)
@app.route('/posters/<path:filename>')
def send_poster(filename):
    return send_from_directory(STATIC_POSTER_PATH, filename)

# Chạy ứng dụng
if __name__ == '__main__':
    print(eval_js("google.colab.kernel.proxyPort(5000)"))
    app.run(port=5000)


https://tu4ggyyzkp-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Oct/2024 14:30:44] "GET /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Oct/2024 14:30:44] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -


Received movie name: Toy Story
Movie info: {'title': 'Toy Story', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 'poster': '/posters/Toy_Story.jpg'}


INFO:werkzeug:127.0.0.1 - - [31/Oct/2024 14:30:54] "POST /?authuser=0 HTTP/1.1" 200 -


Recommendations: [{'title': 'Toy Story 2', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 'poster': '/posters/Toy_Story_2.jpg'}, {'title': 'Finding Nemo', 'genres': 'Adventure|Animation|Children|Comedy', 'poster': '/posters/Finding_Nemo.jpg'}, {'title': 'Glory', 'genres': 'Drama|War', 'poster': '/posters/Glory.jpg'}, {'title': 'Pinocchio', 'genres': 'Animation|Children|Fantasy|Musical', 'poster': '/posters/Pinocchio.jpg'}, {'title': 'Back to the Future Part III', 'genres': 'Adventure|Comedy|Sci-Fi|Western', 'poster': '/posters/Back_to_the_Future_Part_III.jpg'}, {'title': 'Aladdin', 'genres': 'Adventure|Animation|Children|Comedy|Musical', 'poster': '/posters/Aladdin.jpg'}, {'title': 'Splash', 'genres': 'Comedy|Fantasy|Romance', 'poster': '/posters/Splash.jpg'}, {'title': "A Bug's Life", 'genres': 'Genre data unavailable', 'poster': 'https://image.tmdb.org/t/p/w500/Ah3J9OJVc2CNCuH2zMydXy9fmIC.jpg'}, {'title': 'Antz', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 'pos

...

INFO:werkzeug:127.0.0.1 - - [31/Oct/2024 14:32:28] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -


Received movie name: Ocean's Twelve


INFO:werkzeug:127.0.0.1 - - [31/Oct/2024 14:32:33] "POST /?authuser=0 HTTP/1.1" 200 -


Movie info: {'title': "Ocean's Twelve", 'genres': 'Genre data unavailable', 'poster': 'https://image.tmdb.org/t/p/w500/Ad55M8newGWemFWCsMAkxO3fDwl.jpg'}
Movie 'Ocean's Twelve' not found in user_movie_df
Recommendations: []


INFO:werkzeug:127.0.0.1 - - [31/Oct/2024 14:32:36] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -


In [32]:
import os

print("Files in /content/posters:", os.listdir('/content/posters'))


Files in /content/posters: ['index.html', 'posters']
